In [1]:
pip install pyinstaller

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import requests
import json
import pyodbc
from datetime import datetime


In [3]:
def get_auth_token(email, api_key): 
    url_auth = "https://ale.bytebrew.io/auth/generatetoken"
    headers_auth = {
        'Content-Type': 'application/json'
    }
    data_auth = {
        "email": email,
        "apiKey": api_key
    }
    response_auth = requests.post(url_auth, headers=headers_auth, data=json.dumps(data_auth))
    if response_auth.status_code == 200:
        return response_auth.json().get('token')
    else:
        raise Exception(f"Failed to obtain new authentication token: {response_auth.status_code}, {response_auth.text}")


In [4]:
def fetch_data(auth_token, game_id, sdk_key, endpoint, params):
    url = f"https://ale.bytebrew.io/api/gamedata/analytics/{endpoint}"
    headers = {
        'Authorization': f'Bearer {auth_token}'
    }
    response = requests.get(url, headers=headers, params=params)
    print(f"Fetching data from {url} with params {params}")
    if response.status_code == 200:
        data = response.json().get("data", [])
        print(f"Response for {endpoint} for game {game_id}: {data}")
        return data
    else:
        raise Exception(f"Failed to fetch data from {endpoint} for game {game_id}: {response.status_code}, {response.text}")


In [5]:
def process_data(df, endpoint):
    if endpoint == 'retention':
        if 'day_0' in df.columns:
            df['INSTALL DATE'] = pd.to_datetime(df['created_date']).dt.date
            total_users = df['day_0']
            percentages = {f'DAY {i}': (df[f'day_{i}'] / total_users * 100).round(2) for i in range(8)}
            df = df.assign(**percentages)
            df['USERS'] = df['day_0']
            df = df[['INSTALL DATE', 'USERS'] + [f'DAY {i}' for i in range(8)]]
            numeric_columns = [f'DAY {i}' for i in range(8)]
            df[numeric_columns] = df[numeric_columns].applymap(lambda x: f"{x:.2f}%" if pd.notnull(x) else "-")
            df = df.iloc[:1].copy()  
    elif endpoint in ['userdata', 'sessions', 'sessionlength', 'playtime', 'dau', 'progressiondata']:
        print(f"Processing data for endpoint: {endpoint}")
        if 'Date' in df.columns:
            df['DATE'] = pd.to_datetime(df['Date'], format='%Y-%m-%d').dt.date
            if df.columns.size > 1:
                df = df.rename(columns={df.columns[1]: 'COUNT'})
            if 'DATE' in df.columns and 'COUNT' in df.columns:
                df = df[['DATE', 'COUNT']]
            else:
                print(f"Expected columns 'DATE' and 'COUNT' not found in the data for endpoint {endpoint}.")
        else:
            print(f"'Date' column not found in the data for endpoint {endpoint}.")
    else:
        raise ValueError("Unknown endpoint type")

    return df


In [6]:
# Set up credentials and paths
email = "###########"
api_key = "###################"
excel_file_path = "##########"  
start_date = "2024-07-11"
end_date = "2024-07-20"
endpoints = ['retention', 'userdata', 'sessions', 'sessionlength', 'playtime', 'dau', 'progressiondata']

# Load Excel file
try:
    games_df = pd.read_excel(excel_file_path)
except Exception as e:
    print(f"Error loading Excel file: {e}")
    exit()

# Dictionary to store DataFrames for each endpoint
data_frames = {}

try:
    auth_token = get_auth_token(email, api_key)
    for endpoint in endpoints:
        all_data = []
        for index, row in games_df.iterrows():
            game_id = row['game_id']
            sdk_key = row['sdk_key']
            game_name = row['game_name']

            params = {
                'startDate': start_date,
                'endDate': end_date
            }
            if game_id:
                params['gameId'] = game_id
            if sdk_key:
                params['sdkKey'] = sdk_key

            data = fetch_data(auth_token, game_id, sdk_key, endpoint, params)
            if data:
                df = pd.DataFrame(data)

                if not df.empty:
                    df = process_data(df, endpoint)
                    df.insert(0, 'GAME NAME', game_name)
                    df.insert(0, 'GAME ID', game_id)
                    all_data.append(df)

        if all_data:
            combined_df = pd.concat(all_data, ignore_index=True)
            data_frames[endpoint] = combined_df
            
        else:
            print(f"No data available for {endpoint} endpoint.")
except Exception as e:
    print(e)
    exit()


Fetching data from https://ale.bytebrew.io/api/gamedata/analytics/retention with params {'startDate': '2024-07-11', 'endDate': '2024-07-20', 'gameId': '1__szIOoh', 'sdkKey': 'nLTmpcdwfVZDKz9LEUub+BaXQBjvKs3DSS0QphozJxVPsMc6ijLdMOf8rAkLYwGI'}
Response for retention for game 1__szIOoh: [{'created_date': '2024-07-11', 'day_0': 234, 'day_1': 34, 'day_2': 10, 'day_3': 8, 'day_4': 5, 'day_5': 2, 'day_6': 0, 'day_7': 2}, {'created_date': '2024-07-12', 'day_0': 240, 'day_1': 29, 'day_2': 9, 'day_3': 9, 'day_4': 5, 'day_5': 2, 'day_6': 5, 'day_7': 1}, {'created_date': '2024-07-13', 'day_0': 282, 'day_1': 30, 'day_2': 12, 'day_3': 9, 'day_4': 4, 'day_5': 2, 'day_6': 3, 'day_7': 0}, {'created_date': '2024-07-14', 'day_0': 291, 'day_1': 32, 'day_2': 12, 'day_3': 12, 'day_4': 4, 'day_5': 7, 'day_6': 0, 'day_7': 0}, {'created_date': '2024-07-15', 'day_0': 241, 'day_1': 33, 'day_2': 12, 'day_3': 8, 'day_4': 3, 'day_5': 0, 'day_6': 0, 'day_7': 0}, {'created_date': '2024-07-16', 'day_0': 290, 'day_1': 

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15632\1990109159.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[numeric_columns] = df[numeric_columns].applymap(lambda x: f"{x:.2f}%" if pd.notnull(x) else "-")


Fetching data from https://ale.bytebrew.io/api/gamedata/analytics/retention with params {'startDate': '2024-07-11', 'endDate': '2024-07-20', 'gameId': '1yZuhD3In', 'sdkKey': '68pTcH++qSB+pVv6qL4tPyLZByo3ZVEBIgVN2cJU27/YTZrT5rdbU6XXus50Ldbr'}
Response for retention for game 1yZuhD3In: [{'created_date': '2024-07-11', 'day_0': 1, 'day_1': 0, 'day_2': 0, 'day_3': 0, 'day_4': 0, 'day_5': 0, 'day_6': 0, 'day_7': 0}, {'created_date': '2024-07-12', 'day_0': 1, 'day_1': 0, 'day_2': 0, 'day_3': 0, 'day_4': 0, 'day_5': 0, 'day_6': 0, 'day_7': 0}, {'created_date': '2024-07-14', 'day_0': 1, 'day_1': 0, 'day_2': 0, 'day_3': 0, 'day_4': 0, 'day_5': 0, 'day_6': 0, 'day_7': 0}, {'created_date': '2024-07-16', 'day_0': 1, 'day_1': 0, 'day_2': 0, 'day_3': 0, 'day_4': 0, 'day_5': 0, 'day_6': 0, 'day_7': 0}, {'created_date': '2024-07-17', 'day_0': 1, 'day_1': 0, 'day_2': 0, 'day_3': 0, 'day_4': 0, 'day_5': 0, 'day_6': 0, 'day_7': 0}, {'created_date': '2024-07-19', 'day_0': 1, 'day_1': 0, 'day_2': 0, 'day_3'

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15632\1990109159.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[numeric_columns] = df[numeric_columns].applymap(lambda x: f"{x:.2f}%" if pd.notnull(x) else "-")


Fetching data from https://ale.bytebrew.io/api/gamedata/analytics/retention with params {'startDate': '2024-07-11', 'endDate': '2024-07-20', 'gameId': '31cfvBvJh', 'sdkKey': 'ehBvmZaYExJQRERAo8B7kmtcYvrrk/fLWV7OWggyrJHGPsV67BMJ71/amUFV3hzE'}
Response for retention for game 31cfvBvJh: [{'created_date': '2024-07-11', 'day_0': 173, 'day_1': 27, 'day_2': 18, 'day_3': 15, 'day_4': 5, 'day_5': 4, 'day_6': 3, 'day_7': 4}, {'created_date': '2024-07-12', 'day_0': 184, 'day_1': 27, 'day_2': 12, 'day_3': 14, 'day_4': 4, 'day_5': 7, 'day_6': 6, 'day_7': 9}, {'created_date': '2024-07-13', 'day_0': 231, 'day_1': 43, 'day_2': 21, 'day_3': 17, 'day_4': 9, 'day_5': 5, 'day_6': 7, 'day_7': 0}, {'created_date': '2024-07-14', 'day_0': 314, 'day_1': 67, 'day_2': 30, 'day_3': 14, 'day_4': 13, 'day_5': 15, 'day_6': 0, 'day_7': 0}, {'created_date': '2024-07-15', 'day_0': 584, 'day_1': 128, 'day_2': 59, 'day_3': 34, 'day_4': 28, 'day_5': 0, 'day_6': 0, 'day_7': 0}, {'created_date': '2024-07-16', 'day_0': 557, 

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15632\1990109159.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[numeric_columns] = df[numeric_columns].applymap(lambda x: f"{x:.2f}%" if pd.notnull(x) else "-")


Fetching data from https://ale.bytebrew.io/api/gamedata/analytics/retention with params {'startDate': '2024-07-11', 'endDate': '2024-07-20', 'gameId': '5JLOcHrnT', 'sdkKey': 'pxQj6isVpMkeWnJ81h4sokB2mjUa/C8pK/3hwq/GthRhfH02o0s+WAMp1N4DYSQ8'}
Response for retention for game 5JLOcHrnT: [{'created_date': '2024-07-11', 'day_0': 14697, 'day_1': 1410, 'day_2': 454, 'day_3': 281, 'day_4': 197, 'day_5': 119, 'day_6': 129, 'day_7': 92}, {'created_date': '2024-07-12', 'day_0': 21542, 'day_1': 1920, 'day_2': 684, 'day_3': 388, 'day_4': 285, 'day_5': 226, 'day_6': 164, 'day_7': 162}, {'created_date': '2024-07-13', 'day_0': 17851, 'day_1': 1590, 'day_2': 540, 'day_3': 313, 'day_4': 219, 'day_5': 186, 'day_6': 151, 'day_7': 0}, {'created_date': '2024-07-14', 'day_0': 13373, 'day_1': 1036, 'day_2': 373, 'day_3': 241, 'day_4': 160, 'day_5': 132, 'day_6': 0, 'day_7': 0}, {'created_date': '2024-07-15', 'day_0': 17688, 'day_1': 1385, 'day_2': 516, 'day_3': 341, 'day_4': 202, 'day_5': 0, 'day_6': 0, 'day_

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15632\1990109159.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[numeric_columns] = df[numeric_columns].applymap(lambda x: f"{x:.2f}%" if pd.notnull(x) else "-")


Fetching data from https://ale.bytebrew.io/api/gamedata/analytics/userdata with params {'startDate': '2024-07-11', 'endDate': '2024-07-20', 'gameId': '1__szIOoh', 'sdkKey': 'nLTmpcdwfVZDKz9LEUub+BaXQBjvKs3DSS0QphozJxVPsMc6ijLdMOf8rAkLYwGI'}
Response for userdata for game 1__szIOoh: [{'Users': 2312}]
Processing data for endpoint: userdata
'Date' column not found in the data for endpoint userdata.
Fetching data from https://ale.bytebrew.io/api/gamedata/analytics/userdata with params {'startDate': '2024-07-11', 'endDate': '2024-07-20', 'gameId': '1yZuhD3In', 'sdkKey': '68pTcH++qSB+pVv6qL4tPyLZByo3ZVEBIgVN2cJU27/YTZrT5rdbU6XXus50Ldbr'}
Response for userdata for game 1yZuhD3In: [{'Users': 6}]
Processing data for endpoint: userdata
'Date' column not found in the data for endpoint userdata.
Fetching data from https://ale.bytebrew.io/api/gamedata/analytics/userdata with params {'startDate': '2024-07-11', 'endDate': '2024-07-20', 'gameId': '31cfvBvJh', 'sdkKey': 'ehBvmZaYExJQRERAo8B7kmtcYvrrk/f

In [7]:
import pyodbc

# SQL Server connection credentials
server = '##################'
database = '##################'

# Create connection string
conn_str = (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    f'SERVER={server};'
    f'DATABASE={database};'
    'Trusted_Connection=yes;'
)

# Function to insert data into SQL Server
def insert_data_to_sql(df, table_name):
    try:
        # Connect to the SQL Server database
        conn = pyodbc.connect(conn_str)
        print(f"Connected successfully to insert data into {table_name}!")
        
        # Creating a cursor object
        cursor = conn.cursor()

        column_names = df.columns.tolist()
        data_types = [str(df[col].dtype) for col in column_names]

        # Creating the table schema dynamically
        create_table_sql = f"CREATE TABLE {table_name} ("
        for col, dtype in zip(column_names, data_types):
            col = f"[{col}]"
            if dtype == 'object':
                create_table_sql += f"{col} VARCHAR(255),"
            elif dtype == 'int64':
                create_table_sql += f"{col} INT,"
            elif dtype == 'float64':
                create_table_sql += f"{col} FLOAT,"
            else:
                create_table_sql += f"{col} VARCHAR(255),"
        create_table_sql = create_table_sql.rstrip(',') + ")"
        
        # Execute the table creation SQL
        cursor.execute(create_table_sql)
        print(f"Table {table_name} created successfully.")

        # Insert data into the table from the DataFrame
        insert_sql = f"INSERT INTO {table_name} ({', '.join([f'[{col}]' for col in column_names])}) VALUES ({', '.join(['?' for _ in column_names])})"
        for index, row in df.iterrows():
            cursor.execute(insert_sql, tuple(row))
        
        # Commit the changes
        conn.commit()
        print(f"Data for {table_name} inserted successfully.")

    except Exception as e:
        print(f"Error inserting data into table {table_name}: {e}")
    
    finally:
        # Close the cursor and connection
        cursor.close()
        conn.close()

# Insert data into all tables
for endpoint, df in data_frames.items():
    table_name = f"{endpoint}_data"
    insert_data_to_sql(df, table_name)

print("Data insertion complete.")


Connected successfully to insert data into retention_data!
Table retention_data created successfully.
Data for retention_data inserted successfully.
Connected successfully to insert data into userdata_data!
Table userdata_data created successfully.
Data for userdata_data inserted successfully.
Connected successfully to insert data into sessions_data!
Table sessions_data created successfully.
Data for sessions_data inserted successfully.
Connected successfully to insert data into sessionlength_data!
Table sessionlength_data created successfully.
Data for sessionlength_data inserted successfully.
Connected successfully to insert data into playtime_data!
Table playtime_data created successfully.
Data for playtime_data inserted successfully.
Connected successfully to insert data into dau_data!
Table dau_data created successfully.
Data for dau_data inserted successfully.
Connected successfully to insert data into progressiondata_data!
Table progressiondata_data created successfully.
Data fo